<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# PyTorch에서 MNIST 데이터 로드
transform = transforms.Compose([transforms.ToTensor()])
mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
data_loader = torch.utils.data.DataLoader(mnist_data, batch_size=1000, shuffle=True)

In [ ]:
imgs = []

for img, label in data_loader:
    # label이 0인 이미지만 처리합니다.
    # img = img[label == 0]
    img = img.flatten(1)
    imgs.append(img.numpy())

imgs = np.concatenate(imgs, axis=0)

print(imgs.shape)

In [ ]:
# 축소 및 밀도 추정
pca2 = PCA(n_components=2)
data_pca2 = pca2.fit_transform(np.stack(imgs))
data_pca2.shape

kde = gaussian_kde(data_pca2.T)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 2차원 그리드 생성
x = np.linspace(data_pca2[:, 0].min(), data_pca2[:, 0].max(), 100)
y = np.linspace(data_pca2[:, 1].min(), data_pca2[:, 1].max(), 100)
X, Y = np.meshgrid(x, y)

# 그리드 상의 각 지점에서 밀도 계산
positions = np.vstack([X.ravel(), Y.ravel()])
density = kde(positions).reshape(X.shape)

# 3차원 그래프로 시각화
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, density, cmap='viridis', edgecolor='none')
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('Density')
plt.title('Gaussian KDE of MNIST Digits (Label 0) in 3D')
plt.colorbar(surf)
plt.show()

In [ ]:
# pca3 = PCA(n_components=3)
# data_pca3 = pca3.fit_transform(np.stack(imgs))
# data_pca3.shape

# kde = gaussian_kde(data_pca3.T)

# # 3D 그리드 생성
# x = np.linspace(data_pca3[:, 0].min(), data_pca3[:, 0].max(), 100)
# y = np.linspace(data_pca3[:, 1].min(), data_pca3[:, 1].max(), 100)
# z = np.linspace(data_pca3[:, 2].min(), data_pca3[:, 2].max(), 100)
# X, Y, Z = np.meshgrid(x, y, z)
# positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])

# # 밀도 계산
# density = kde(positions).reshape(X.shape)

# # 밀도가 높은 지점만 시각화하기 위한 임계값 설정
# threshold = density.mean()

# # 3D 그래프 시각화
# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(X[density > threshold], Y[density > threshold], Z[density > threshold], c=density[density > threshold], cmap='viridis')
# plt.title('Gaussian KDE of MNIST Digits (Label 0)')
# plt.show()

In [ ]:
# 데이터를 라벨별로 분리
label_data = {i: [] for i in range(10)}
for img, label in zip(images, labels):
    label_data[label.item()].append(img)

In [ ]:
label_data.keys() # 라벨이 key값 해당하는 데이터들의 리스트가 value인 딕셔너리.

In [ ]:
for label in label_data:
    print(img)

In [ ]:
# 라벨별로 PCA와 KDE 수행
pca_results = {}
kde_results = {}
for label in label_data:
    # 라벨별 데이터에 대해 PCA 수행
    pca = PCA(n_components=3)
    data_pca = pca.fit_transform(np.stack(label_data[label]))
    pca_results[label] = data_pca

    # 라벨별 데이터에 대해 KDE 수행
    kde = gaussian_kde(data_pca.T)
    kde_results[label] = kde

In [ ]:
# 3차원 그리드 생성
x = np.linspace(-7, 7, 100) # 예상 데이터 범위 설정
y = np.linspace(-7, 7, 100)
z = np.linspace(-7, 7, 100)
X, Y, Z = np.meshgrid(x, y, z)
xyz = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 3D 그래프 시각화
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# 색상 맵 선택
cmap = plt.get_cmap('viridis')

# 3D 그래프 시각화 준비
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# 라벨별로 밀도 계산 및 시각화
for label in pca_results:
    data_pca = pca_results[label]
    kde = kde_results[label]

    # 해당 라벨의 점들에 대한 밀도 계산
    density = kde(data_pca.T)

    # 밀도 값을 색상으로 변환 (0-1 사이로 정규화)
    colors = cmap((density - density.min()) / (density.max() - density.min()))

    # 라벨별로 시각화
    ax.scatter(data_pca[:, 0], data_pca[:, 1], data_pca[:, 2], c=colors, label=str(label), edgecolor='w')

# 축 라벨 및 제목 설정
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('3D Gaussian KDE of MNIST Data by Label')

# 범례 표시
ax.legend()

# 그래프 표시
plt.show()


In [ ]:
# PCA를 사용하여 데이터 차원 축소
pca = PCA(n_components=3)
data_pca = pca.fit_transform(images)

In [ ]:
data_pca.shape

In [ ]:
# 가우시안 KDE로 밀도 함수 추정
kde = gaussian_kde(data_pca.T) # kde는 특성이 행을로 구성된 데이터를 입력으로한다.

# 3차원 그리드에서 밀도 계산
x = np.linspace(data_pca[:,0].min(), data_pca[:,0].max(), 100)
y = np.linspace(data_pca[:,1].min(), data_pca[:,1].max(), 100)
z = np.linspace(data_pca[:,2].min(), data_pca[:,2].max(), 100)
X, Y, Z = np.meshgrid(x, y, z)
xyz = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])
density = kde(xyz).reshape(X.shape)

# 밀도 기준으로 데이터 포인트 색상 결정
colors = density - density.min()
colors = colors / colors.max()

# 3D 그래프 시각화
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X.ravel(), Y.ravel(), Z.ravel(), c=colors, cmap='viridis', marker='o')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('3D Gaussian KDE of MNIST Data')
plt.show()